<a href="https://colab.research.google.com/github/wizard339/DLSchool/blob/main/time_rows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание погоды
## Coding Stream 

Что мы хотим сделать?  
У нас есть данные по погоде за несколько лет со многих метеостанций.  
Мы попытаемся предсказать погоду на следующий день по предыдущим N дням с помощью линейной регрессии.

Для начала качаем данные с ftp.

In [1]:
import shutil
import urllib
from contextlib import closing

with closing(urllib.request.urlopen('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2018.csv.gz')) as r:
    with open('./2018.csv.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

with closing(urllib.request.urlopen('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2016.csv.gz')) as r:
    with open('./2016.csv.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

with closing(urllib.request.urlopen('ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/2017.csv.gz')) as r:
    with open('./2017.csv.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

Файлы довольно большие, поэтому мы не хотим их считывать с помощью pandas. Вместо этого мы используем средства самого Python.

Формат следующий. Файл csv, каждая строка такая:   
`<id_станции>, <дата>, <метрика>, <значение>, <другие_параметры>`  
Нас интересуют первые 4 столбца

Мы прочитаем все интересующие нас данные в память и выберем те метеостанции, которые вернули показания по средней дневной температуре и направлению ветра в каждый из дней года.

Dask – альтернатива pandas, позволяет делать многопоточные вычисления.

Еще есть Spark/PySpark, hdfs (библиотека – hdf5)

In [2]:
import gzip
with gzip.open('./2018.csv.gz', 'rb') as f_in:
    with open('2018.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('./2016.csv.gz', 'rb') as f_in:
    with open('2016.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open('./2017.csv.gz', 'rb') as f_in:
    with open('2017.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [3]:
import pandas as pd

In [4]:
%time data_2018 = pd.read_csv("./2018.csv", header=None)

CPU times: user 24.8 s, sys: 15.5 s, total: 40.3 s
Wall time: 41.4 s


In [5]:
!head -10 2018.csv

AE000041196,20180101,TMAX,259,,,S,
AE000041196,20180101,TMIN,112,,,S,
AE000041196,20180101,TAVG,186,H,,S,
AEM00041194,20180101,TMAX,250,,,S,
AEM00041194,20180101,PRCP,0,,,S,
AEM00041194,20180101,TAVG,209,H,,S,
AEM00041217,20180101,TMIN,132,,,S,
AEM00041217,20180101,TAVG,191,H,,S,
AEM00041218,20180101,TAVG,193,H,,S,
AFM00040938,20180101,TAVG,82,H,,S,


## Необходимый код

In [6]:
import sys
from sklearn import linear_model


def days_in_year(year):
    if year % 4 != 0:
        return 365
    if year % 400 == 0:
        return 366
    if year % 100 == 0:
        return 365
    return 366


def parse_years(years):
    """
    codes = {
        code: {
            'dt': {
                'TAVG': float,
                'SNWD': float
            }
        }
    }
    
    codes = {
        'AEM00041218': {
            '20160101': {
                'TAVG': -5,
                'SNWD': 0
            },
            '20160102' : {
                ...
            }
        },
    }
    
    result = {
        code: [
            (temp1, wind1),
            (temp2, wind2),
            ...
        ]
    }
    """
    result = {}
    for year in years:
        codes = {}
        for line in open('{year}.csv'.format(year=year), 'rt'):
            if line and line[-1] == '\n':
                line = line[:-1]
            data = line.split(',')
            code, dt, metric, value = data[0:4]
            if metric not in ['TAVG', 'SNWD']:
                continue
            if code not in codes:
                codes[code] = {}
            dates = codes[code]
            if dt not in dates:
                dates[dt] = {}
            dates[dt][metric] = float(value) / 10
        for code, dates in codes.items():
            data_temp = [dates[dt]['TAVG'] for dt in sorted(dates.keys()) if 'TAVG' in dates[dt]]
            data_wind = [dates[dt]['SNWD'] for dt in sorted(dates.keys()) if 'SNWD' in dates[dt]]
            if len(data_temp) != days_in_year(year) or len(data_wind) != days_in_year(year):
                continue
            if code not in result:
                result[code] = list(zip(data_temp, data_wind))
            else:
                result[code] += list(zip(data_temp, data_wind))
    return result

In [7]:
def iterate_data(data, window):
    """
    Сюда приходит result
    
    yield создает итератор, который при каждом вызове (или итерации по нему) возвращает один объект за раз
    Это полезно, когда у нас большие файлы, которые мы не хотим грузить в память
    """
    for _, items in data.items():
        for i in range(0, len(items) - window - 1):
             yield items[i : i + window], items[i + window][0]

def calc_mse(data, window, model):
    result, count = 0.0, 0
    for x, y in iterate_data(data, window):
        result += ((model(x) - y)**2)
        count += 1
    return result / count


def prepare_features(x):
    return [x[i][0] for i in range(len(x))] + [x[i][1] for i in range(len(x))]

In [8]:
def train_linear_regression(data, window):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    return regr


def apply_regression(data, model):
    return model.predict([prepare_features(data)])[0]


def baseline_model(data):
    # data is a list of tuples (temp, wind)
    return data[-1][0]

## Чтение данных

Прочитаем наши данные  
Данные для обучения и валидации не должны пересекаться – иначе это прямой путь к переобучению

In [9]:
train = parse_years([2016])
test = parse_years([2017])

Зададим окно, с которым мы будем обучать модель. Это будет количество дней (признаков/2), на которые мы будем смотреть

In [10]:
window = 14

## Baseline-модель

Посчитаем метрики на обучении и валидации для нашей baseline-модели.  
В качестве базовой модели выберем такую - будем всегда возвращать температуру за вчерашний день.  

Почему мы делаем так: наша задача состоит в том, чтобы предсказать температуру на следующий день.  
В качестве такого предсказания мы будем возвращать температуру за вчерашний день, так как обычно изменение температуры за соседние дни бывает не очень сильное.

In [11]:
calc_mse(train, window, baseline_model)

8.635652978002812

In [12]:
calc_mse(test, window, baseline_model)

8.830649009826294

## Линейная регрессия

Давайте обучим нашу модель линейной регрессии:

In [13]:
model = train_linear_regression(train, window)

И посмотрим на результаты:

In [14]:
calc_mse(train, window, lambda x: apply_regression(x, model))

7.205531465193543

In [15]:
calc_mse(test, window, lambda x: apply_regression(x, model))

7.70706933578011

Получилось, что линейная регрессия справляется лучше, чем baseline-модель.  
Также, метрики не сильно отличаются между обучением и валидацией. Это значит, что переобучения не произошло.

Что, если мы возьмем другой размер окна?

In [16]:
window = 30
model = train_linear_regression(train, window)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.079836812835604
MSE test 7.664089839760451


## Регуляризация

Метрика на валидации заметно выше, чем на обучении. Похоже, мы встретились с переобучением.  
Попробуем применить регуляризацию:

In [17]:
from sklearn.linear_model import Ridge

def train_ridge_regression(data, window, alpha=1.0):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = Ridge(alpha=alpha)
    regr.fit(X, Y)
    return regr

In [18]:
window = 30
model = train_ridge_regression(train, window, alpha=10)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.079836813121136
MSE test 7.664090321867854


In [19]:
window = 7
model = train_ridge_regression(train, window, alpha=100)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 7.4263111126158154
MSE test 7.856654630158888


## Random Forest

Давайте попробуем случайный лес (осторожно, обучается намного дольше регрессии):

In [20]:
from sklearn.ensemble import RandomForestRegressor

def train_random_forest_regression(data, window, n_estimators=10, max_depth=None):
    X, Y = [], []
    for x, y in iterate_data(data, window):
        X.append(prepare_features(x))
        Y.append(y)
    regr = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    regr.fit(X, Y)
    return regr

In [21]:
window = 30
model = train_random_forest_regression(train, window)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 0.7299032598806909
MSE test 9.210695129740861


Видим, что модель очень сильно переобучилась. Попробуем ограничить глубину деревьев:

In [22]:
window = 30
model = train_random_forest_regression(train, window, max_depth=2)
print("MSE train", calc_mse(train, window, lambda x: apply_regression(x, model)))
print("MSE test", calc_mse(test, window, lambda x: apply_regression(x, model)))

MSE train 16.53430742071886
MSE test 16.793567823141778


Переобучения больше нет, но результаты даже намного хуже, чем у baseline-модели. По-хорошему тут нужно запускать grid search на разные параметры, но это очень долго.

Над чем еще можно было бы поработать?

 * Использовать те метеостанции, которые не вернули данные за весь год - улучшит это обучение или ухудшит?
 * Учесть месяц/время года в качестве признаков
 * Учесть геопозицию метеостанции (это есть в соседних файлах на ftp, их можно скачать)
 * Изменить предобработку данных?
 * Что еще?